# Exercise — Image Generation Agent with Cost Approval

This notebook extends the Day 2 tooling lessons by combining **Model Context Protocol (MCP)** integrations with **long-running operations**. I will build an agent that generates images while enforcing a human approval step for bulk (high-cost) requests.

## Scenario

Build an agent that uses an MCP image generation server. The agent must:

- ✅ Auto-approve and generate a **single** image immediately.
- ⏸️ Pause and request approval when the user asks for **more than one** image.
- 🔁 Resume once approval is provided and generate the requested images.
- 🔍 Encourage experimentation with different public MCP image servers.

We'll reuse the experimental [`@modelcontextprotocol/server-everything`](https://github.com/modelcontextprotocol/servers/tree/main/src/everything) as a placeholder image server. Swap it for another MCP image server for testing later.

In [ ]:
import os

try:
    from kaggle_secrets import UserSecretsClient  # Available inside Kaggle notebooks
except ImportError:
    UserSecretsClient = None

if UserSecretsClient is not None:
    try:
        GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        print("✅ Setup and authentication complete.")
    except Exception as e:
        print(
            "🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details:",
            e,
        )
else:
    print(
        "ℹ️ kaggle_secrets not available. Set the GOOGLE_API_KEY environment variable manually before running the notebook."
    )

In [ ]:
import uuid
import base64
from IPython.display import display, Image as IPImage

from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters
from google.adk.tools.tool_context import ToolContext
from google.adk.apps.app import App, ResumabilityConfig

print("✅ ADK components imported successfully.")

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

## 2. Connect to an MCP Image Server

We use the experimental `server-everything` MCP server which exposes a `getTinyImage` tool.

In [ ]:
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP toolset ready.")

## 3. Approval Gate for Bulk Image Generation

The function tool below encapsulates the approval policy. Requests for one image are allowed immediately. Larger requests trigger a confirmation pause via `tool_context.request_confirmation()`.

In [ ]:
BULK_THRESHOLD = 1  # Requests larger than this require approval


def evaluate_image_request(prompt: str, count: int, tool_context: ToolContext) -> dict:
    '''Applies approval policy for image generation.'''

    if count <= BULK_THRESHOLD:
        return {
            "status": "approved",
            "approval_type": "auto",
            "prompt": prompt,
            "count": count,
        }

    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=(
                f"⚠️ Bulk image request detected (count={count}). "
                "Approve before generating multiple images?"
            ),
            payload={"prompt": prompt, "count": count},
        )
        return {
            "status": "pending",
            "prompt": prompt,
            "count": count,
        }

    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "approval_type": "manual",
            "prompt": prompt,
            "count": count,
        }

    return {
        "status": "rejected",
        "prompt": prompt,
        "count": count,
        "message": "Bulk request rejected by approver.",
    }


print("✅ Approval function tool defined.")

## 4. Build the Agent, App, and Runner

The agent first evaluates requests with `evaluate_image_request`. For approved requests it calls the MCP tool, summarises results, and displays Base64 images. The resumable app allows the workflow to pause for approvals and later resume seamlessly.

In [ ]:
image_agent = LlmAgent(
    name="image_approval_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=(
        "You are an assistant that generates images via MCP tools while respecting approval policies.

"
        "Workflow:
"
        "1. Always start by calling evaluate_image_request(prompt=..., count=...).
"
        "2. If the tool returns status "pending", inform the user that approval is required and wait.
"
        "3. If the tool returns status "rejected", explain that the request was denied.
"
        "4. If approved, call the getTinyImage MCP tool once per requested image.
"
        "5. After each call, note the tool output and provide a final summary including how many images were generated.
"
        "6. Provide Base64 image data or rely on helper functions to render it."
    ),
    tools=[
        FunctionTool(func=evaluate_image_request),
        mcp_image_server,
    ],
)

image_app = App(
    name="image_generation_app",
    root_agent=image_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

session_service = InMemorySessionService()
image_runner = Runner(app=image_app, session_service=session_service)

print("✅ Agent, app, and runner created.")

## 5. Helper Utilities

Support functions to inspect events, detect approval pauses, resume execution, and render returned images.

In [ ]:
def check_for_approval(events):
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None


def create_approval_response(approval_info, approved):
    confirmation = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(role="user", parts=[types.Part(function_response=confirmation)])


def print_agent_text(events):
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")


def display_images_from_events(events):
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.function_response:
                    response = part.function_response.response
                    for item in response.get("content", []):
                        if item.get("type") == "image":
                            data = base64.b64decode(item["data"])
                            display(IPImage(data=data))


print("✅ Helper functions ready.")

##  6. Workflow Execution

The `run_image_workflow()` helper encapsulates the pause/resume logic for long-running operations.

In [ ]:
async def run_image_workflow(prompt: str, count: int, auto_approve: bool = True):
    print("
" + "=" * 72)
    print(f"User > Generate {count} image(s) for prompt: '{prompt}'
")

    session_id = f"img_{uuid.uuid4().hex[:8]}"
    await session_service.create_session(
        app_name=image_app.name, user_id="demo_user", session_id=session_id
    )

    request = types.Content(
        role="user",
        parts=[types.Part(text=f"Generate {count} image(s) of {prompt}.")],
    )

    events = []
    async for event in image_runner.run_async(
        user_id="demo_user", session_id=session_id, new_message=request
    ):
        events.append(event)

    approval_info = check_for_approval(events)

    if approval_info:
        print("⏸️  Awaiting approval for bulk image request...")
        decision_text = "APPROVE ✅" if auto_approve else "REJECT ❌"
        print(f"🤔 Human Decision: {decision_text}
")

        async for event in image_runner.run_async(
            user_id="demo_user",
            session_id=session_id,
            new_message=create_approval_response(approval_info, auto_approve),
            invocation_id=approval_info["invocation_id"],
        ):
            events.append(event)

    print_agent_text(events)
    display_images_from_events(events)
    print("=" * 72 + "
")


print("✅ Workflow helper defined.")

## 7. Demo Runs

Test both approval paths:

1. Auto-approved single image request.
2. Bulk request that we approve.
3. Bulk request that we reject.

In [ ]:
await run_image_workflow("sunset over mountains", count=1, auto_approve=True)
await run_image_workflow("futuristic skyline", count=3, auto_approve=True)
await run_image_workflow("abstract sculpture", count=4, auto_approve=False)